In [1]:
import numpy as np
import pandas as pd

In [2]:
# 构造判断矩阵
def build_matrix(values):
    n = len(values)
    mat = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            if i == j:
                mat[i][j] = 1
            else:
                mat[i][j] = values[i] / values[j]
                mat[j][i] = 1 / mat[i][j]
    return mat

In [3]:
# 计算每个准则的权重
def calc_weight(mat):
    eig_val, eig_vec = np.linalg.eig(mat)
    max_eig_val_index = np.argmax(eig_val)
    max_eig_vec = eig_vec[:, max_eig_val_index]
    weight = max_eig_vec / max_eig_vec.sum()
    return weight

In [4]:
# 层次分析法主程序
def ahp(data):
    # 构造层次结构
    criteria = data.columns.tolist()  # 准则层
    alternatives = data.index.tolist()  # 方案层
    n = len(criteria) + len(alternatives)  # 层次总数
    hierarchy = [alternatives, criteria]
     # 计算判断矩阵
    matrices = []
    for i in range(len(criteria)):
        mat = build_matrix(data[criteria[i]].tolist())
        matrices.append(mat)
    matrices.append(np.eye(len(alternatives)))  # 方案层的判断矩阵为单位矩阵

    # 计算每个准则的权重
    weights = []
    for mat in matrices:
        weights.append(calc_weight(mat))
    criteria_weights = weights[:-1]
    alternatives_weights = weights[-1]

    # 计算每个方案的总权重
    total_weights = []
    for i in range(len(alternatives)):
        total_weight = 0
        for j in range(len(criteria)):
            total_weight += criteria_weights[j][i] * alternatives_weights[i]
        total_weights.append(total_weight)

    # 返回每个方案的总权重
    return pd.DataFrame(total_weights, index=alternatives, columns=['Total Weight'])

In [11]:
import pandas as pd

# 读取数据
data = pd.read_excel('../Resource/ExperimentDataBase/Spad_various_index.xlsx',usecols=['Band1蓝','Band2绿','Band3红','Band4红边','Band5近红外1','Band6近红外2','SPAD','株高','LAI','Gs','含水率'])

# 运行层次分析法
weights = ahp(data)

# 打印结果
print(weights)

          Total Weight
0   0.168939+0.000000j
1   0.000000+0.000000j
2   0.000000+0.000000j
3   0.000000+0.000000j
4   0.000000+0.000000j
..                 ...
61  0.000000+0.000000j
62  0.000000+0.000000j
63  0.000000+0.000000j
64  0.000000+0.000000j
65  0.000000+0.000000j

[66 rows x 1 columns]


In [13]:
# 将重构后的数据保存到DataFrame对象中
df2 = pd.DataFrame(weights)

# 保存DataFrame对象到excel文件中
df2.to_excel('reconstructed_pad.xlsx',index=False)